Due to the limitations in the datasets, webscraping is required to get the additional information.
1. IMDB
    -Movie summary
2. Goodreads
    -Book summary
    -Book genre
Will also explore using the API to build the database.

# Webscraping

In [57]:
# import the necessary libraries
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen
from bs4 import BeautifulSoup
import requests
import urllib2

import pandas as pd
import time
from time import sleep

In [2]:
fileBook1='./data/goodbooks-10k-master/books.csv'
dataBook1=pd.read_csv(fileBook1)

dataBook1.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [339]:
bookid = dataBook1['goodreads_book_id'][0]
title = dataBook1['original_title'][0].replace(" ","-")
print bookid, title
final = str(bookid) + "-" + title
print final

2767052 The-Hunger-Games
2767052-The-Hunger-Games


In [340]:
# starting url is with goodreads website
# sample website link for a book: https://www.goodreads.com/book/show/929-Memoirs_of_a_Geisha
# Note: As good reads updated their website. Will have to use a different method to scrape
url = "https://www.goodreads.com/book/show/"
#download the URL and extract the content to the variable html 
url = url + str(final)

page = requests.get(url)

#specifying a desired format of “page” using the html parser - 
# this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")

In [341]:
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<!DOCTYPE html>
<html class="desktop ">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
  <title>
   The Hunger Games (The Hunger Games, #1) by Suzanne Collins
  </title>
  <script type="text/javascript">
   var ue_t0=window.ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:arguments};c.ueLogError(k);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(fun

In [342]:
#Getting the book summary
entry = soup.find('div', {'id':'description'})
summary = entry.find('span', {'style':'display:none'}).text
summary

u"Winning will make you famous. Losing means certain death.The nation of Panem, formed from a post-apocalyptic North America, is a country that consists of a wealthy Capitol region surrounded by 12 poorer districts. Early in its history, a rebellion led by a 13th district against the Capitol resulted in its destruction and the creation of an annual televised event known as the Hunger Games. In punishment, and as a reminder of the power and grace of the Capitol, each district must yield one boy and one girl between the ages of 12 and 18 through a lottery system to participate in the games. The 'tributes' are chosen during the annual Reaping and are forced to fight to the death, leaving only one survivor to claim victory.When 16-year-old Katniss's young sister, Prim, is selected as District 12's female representative, Katniss volunteers to take her place. She and her male counterpart Peeta, are pitted against bigger, stronger representatives, some of whom have trained for this their whol

In [331]:
# looking good so far but the problem is now that it's in unicode..
# We are going to straight away remove the unicode from the string
str(summary)

"Winning will make you famous. Losing means certain death.The nation of Panem, formed from a post-apocalyptic North America, is a country that consists of a wealthy Capitol region surrounded by 12 poorer districts. Early in its history, a rebellion led by a 13th district against the Capitol resulted in its destruction and the creation of an annual televised event known as the Hunger Games. In punishment, and as a reminder of the power and grace of the Capitol, each district must yield one boy and one girl between the ages of 12 and 18 through a lottery system to participate in the games. The 'tributes' are chosen during the annual Reaping and are forced to fight to the death, leaving only one survivor to claim victory.When 16-year-old Katniss's young sister, Prim, is selected as District 12's female representative, Katniss volunteers to take her place. She and her male counterpart Peeta, are pitted against bigger, stronger representatives, some of whom have trained for this their whole

In [343]:
# Getting the book genre
entry = soup.findAll('a', {'class':'actionLinkLite bookPageGenreLink'})
for item in entry:
    print item.text

# In the movie database, the genre appears as "Adventure|Animation|Children|Comedy|Fantasy"
# will do the same here for consistency
text=""
for item in entry:
    text += "|" + item.text

# get the text without the first "|"
print text[1:]

Young Adult
Fiction
Science Fiction
Dystopia
Fantasy
Science Fiction
Young Adult|Fiction|Science Fiction|Dystopia|Fantasy|Science Fiction


In [154]:
# how many rows to go through
print len(dataBook1['goodreads_book_id'])

#going to need a progress bar here
from ipywidgets import FloatProgress
from IPython.display import display

10000


In [277]:
s = "Bridget Jones's Diary"
print s.replace(" ","-").replace("'","-")

Bridget-Jones-s-Diary


In [496]:
from random import randint

#Creating the function to scrape through the website
url = "https://www.goodreads.com/book/show/"
b_id=[]
b_summary=[]
b_genre=[]
b_titles=[]

#testing for getting the url
for book in dataBook1['goodreads_book_id'][5:10]:
    index = dataBook1.loc[dataBook1['goodreads_book_id']==book].index[0]
    # print index
    # index is to ensure the books are set correctly
    title = dataBook1['original_title'][index].replace(" ","-")
    final = str(book) + "-" + title
    link = url + final
    print link

https://www.goodreads.com/book/show/11870085-The-Fault-in-Our-Stars
https://www.goodreads.com/book/show/5907-The-Hobbit-or-There-and-Back-Again
https://www.goodreads.com/book/show/5107-The-Catcher-in-the-Rye
https://www.goodreads.com/book/show/960-Angels-&-Demons-
https://www.goodreads.com/book/show/1885-Pride-and-Prejudice


In [497]:
#Note to prevent good reads from updating. Will do in batches of 100
#to track progress
f = FloatProgress(min=0, max=500) # instantiate the bar
display(f) # display the bar

for book in dataBook1['goodreads_book_id'][3000:3500]:
    #Index/BookID
    index = dataBook1.loc[dataBook1['goodreads_book_id']==book].index[0]
    b_id.append(book)
    
    #title
    b_titles.append(dataBook1['original_title'][index])
    t = str(dataBook1['original_title'][index])
    title = t.replace(" '","-").replace("'","-")
    final = str(book) + "-" + title
    
    #download the URL and extract the content to the variable html 
    link = url + final
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    
    #Summary
    try:
        entrySummary = soup.find('div', {'id':'description'})
        s = entrySummary.find('span', {'style':'display:none'}).text
        #some of the text could not be decoded properly
        #this workaround is use instead to decode the text
        s=s.encode('ascii', 'ignore').decode('ascii')
        b_summary.append(s)
    except:
        b_summary.append("None")
    
    #Genre
    try:
        entryGenre = soup.findAll('a', {'class':'actionLinkLite bookPageGenreLink'})
        text=""
        for item in entryGenre:
             text += "|" + str(item.text)
        b_genre.append(text[1:])
    except:
        b_genre.append("None")
    
    f.value += 1 # signal to increment the progress bar
    
    #give a six-eight seconds pause between books
    sleep(randint(6,8))

FloatProgress(value=0.0, max=500.0)

In [498]:
b_id2=[]
b_summary2=[]
b_genre2=[]
b_titles2=[]

In [499]:
sleep(randint(30,40))

#Note to prevent good reads from updating. Will do in batches of 100
#to track progress
f = FloatProgress(min=0, max=500) # instantiate the bar
display(f) # display the bar

for book in dataBook1['goodreads_book_id'][3500:4000]:
    #Index/BookID
    index = dataBook1.loc[dataBook1['goodreads_book_id']==book].index[0]
    b_id2.append(book)
    
    #title
    b_titles2.append(dataBook1['original_title'][index])
    t = str(dataBook1['original_title'][index])
    title = t.replace(" '","-").replace("'","-")
    final = str(book) + "-" + title
    
    #download the URL and extract the content to the variable html 
    link = url + final
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    
    #Summary
    try:
        entrySummary = soup.find('div', {'id':'description'})
        s = entrySummary.find('span', {'style':'display:none'}).text
        #some of the text could not be decoded properly
        #this workaround is use instead to decode the text
        s=s.encode('ascii', 'ignore').decode('ascii')
        b_summary2.append(s)
    except:
        b_summary2.append("None")
    
    #Genre
    try:
        entryGenre = soup.findAll('a', {'class':'actionLinkLite bookPageGenreLink'})
        text=""
        for item in entryGenre:
             text += "|" + str(item.text)
        b_genre2.append(text[1:])
    except:
        b_genre2.append("None")
    
    f.value += 1 # signal to increment the progress bar
    
    #give a six-eight seconds pause between books
    sleep(randint(6,8))

FloatProgress(value=0.0, max=500.0)

In [500]:
# test the lists and make sure they match before converting the pd
print b_id[99]
print b_titles[99]
print b_summary[99]
print b_genre[99]
print len(b_id)

322351
How to Eat Fried Worms
Fans of Diary of a Wimpy Kid, The Worst Class Trip Ever, and the Tapper Twins series "will revolt and delight" in How to Eat Fried Worms. Because of a bet, Billy is in the uncomfortable position of having to eat fifteen worms in fifteen days. The worms are supplied by his opponent, whose motto is "The bigger and juicier, the better!" At first Billy's problem is whether or not he can swallow the worm placed before him, even with a choice of condiments from peanut butter to horseradish. But later it looks as if Billy will win, and the challenge becomes getting to the worm to eat it. Billy's family, after checking with the doctor, takes everything in stride. They even help Billy through his gastronomic ordeal, which twists and turns with each new day, leaving the outcome of the bet continually in doubt. This edition also includes an eight-page full color insert with photos from the movie!
Childrens|Fiction|Young Adult
500


In [501]:
# test the lists and make sure they match before converting the pd
print b_id2[99]
print b_titles2[99]
print b_summary2[99]
print b_genre2[99]
print len(b_id2)

9589
Hocus Pocus
None
Fiction|Humor|Science Fiction|Literature
500


In [454]:
#clean up if needed
# b_id = b_id[:-99]
# b_titles = b_titles[:-99]
# b_summary = b_summary[:-99]
# b_genre = b_genre[:-99]
b_titles[74] = "Bridget Jones's Diary"
print b_titles[74]

Bridget Jones's Diary


In [502]:
#convert them to pd
result = [('goodreads_book_id', b_id),
          ('title', b_titles),
          ('summary', b_summary),
          ('genre', b_genre)]

result2 = [('goodreads_book_id', b_id2),
          ('title', b_titles2),
          ('summary', b_summary2),
          ('genre', b_genre2)]

In [503]:
df = pd.DataFrame.from_items(result)

df2 = pd.DataFrame.from_items(result2)

In [373]:
df

,goodreads_book_id,title,summary,genre
0,10799,A Farewell to Arms,A Farewell to Arms is the unforgettable story ...,Classics|Fiction|Historical|Historical Fiction...
1,391729,The Tell-Tale Heart,Edgar Allan Poe remains the unsurpassed master...,Classics|Horror|Fiction|Short Stories|Academic...
2,10534,孫子兵法 [Sūnzi bīngfǎ],"Twenty-Five Hundred years ago, Sun Tzu wrote t...",Nonfiction|Classics|Philosophy|History|War
3,11367726,Defending Jacob,Award-winning author William Landay has writte...,Fiction|Mystery|Thriller|Mystery|Crime|Thrille...
4,4980,Breakfast of Champions,None,Fiction|Classics|Science Fiction|Humor|Literature
5,627206,Drawing on the Right Side of the Brain,When Drawing on the Right Side of the Brain wa...,Art|Nonfiction|Art|Drawing|Reference|Psycholog...
6,5826,Bel Canto,"In an unnamed South American country, a world-...",Fiction|Contemporary|Literary Fiction|Book Club
7,1842,"Guns, Germs, and Steel: The Fates of Human Soc...","""Diamond has written a book of remarkable scop...",History|Nonfiction|Science|Anthropology|Sociology
8,9375,Fried Green Tomatoes at the Whistle Stop Cafe,It's first the story of two women in the 1980s...,Fiction|Historical|Historical Fiction|Lgbt|Wom...
9,37781,Things Fall Apart,"THINGS FALL APART tells two overlapping, inter...",Fiction|Classics|Cultural|Africa|Historical|Hi...


In [504]:
#convert pd to csv
# 0-999: completed
# 1000-2000: completed

df.to_csv('webscrape-goodreads-3000-3500.csv', encoding='utf-8', index=False)
df2.to_csv('webscrape-goodreads-3500-4000.csv', encoding='utf-8', index=False)

# Web API

In [98]:
# using the IMDB api available https://github.com/alberanid/imdbpy
from imdb import IMDb  # to access imdb API

In [99]:
# create an instance of the IMDb class
ia = IMDb()

In [102]:
#test test
# get a movie
movie = ia.get_movie('0133093')
movie.get('title')

u'The Matrix'

In [103]:
#get the plot
movie.get('plot')

[u"Thomas A. Anderson is a man living two lives. By day he is an average computer programmer and by night a hacker known as Neo. Neo has always questioned his reality, but the truth is far beyond his imagination. Neo finds himself targeted by the police when he is contacted by Morpheus, a legendary computer hacker branded a terrorist by the government. Morpheus awakens Neo to the real world, a ravaged wasteland where most of humanity have been captured by a race of machines that live off of the humans' body heat and electrochemical energy and who imprison their minds within an artificial reality known as the Matrix. As a rebel against the machines, Neo must return to the Matrix and confront the agents: super-powerful computer programs devoted to snuffing out Neo and the entire human rebellion.::redcommander27",
 u"Thomas A. Anderson ids a man leading a double life. By day, he isu an average computer programmer who pays his taxes and helps his landlady take out the garbage. At night, he

In [ ]:
# grabbing the database with imdb id
dataMovie='./data/ml-latest/links.csv'
dataMovie=pd.read_csv(dataMovie)

In [ ]:
# create an instance of the IMDb class
ia = IMDb()

In [ ]:
for item in dataMovie["imdbId"][5000:5500]:
    #the imdbID
    item = int(item)
    imdbId.append(item)
    
    try:
        #the movie title 
        movie = ia.get_movie(item)
        mtitle = movie.get('title')
        m_title.append(mtitle)
    except:
        m_title.append("None")
        
    try:
        #movie summary
        plot = movie.get('plot')[0]
        plot = plot.split("::")
        plot = str(plot[0])
        m_summary.append(plot)
    except:
        m_summary.append("None")

In [ ]:
#convert them to pd
result = [('imdbId', imdbId),
          ('title', m_title),
          ('summary', m_summary)]

m_df = pd.DataFrame.from_items(result)

In [ ]:
m_df.to_csv('api-imdb-movie_summary_5000_5500.csv', encoding='utf-8', index=False)